# Retrieve notebook and CPU tables from Baserow

PH, Feb 2025

In [1]:
import requests
import pandas as pd

Baserow API access token (read only access to all tables of the *Computers* project

In [2]:
BASEROW_ACCESS_TOKEN='3dDIsaSBqeszii1EmBJ5FSKVPSJmDms8'

Table IDs

In [3]:
NB_OFFERS_TABLE = '444311'
CPUS_TABLE = '444310'

In [4]:
def get_baserow_rows(TABLE_ID, TOKEN, page=1, size=100):
    """Read rows of table `TABLE_ID`.
    
    Returns (row count, json 'results' field)

    If API error, returns the json error message.
    """
    params = {
        'page': int(page),
        'size': int(size),
    }
    res = requests.get(
        f'https://api.baserow.io/api/database/rows/table/{TABLE_ID}/?user_field_names=true',
        headers={
            'Authorization': f'Token {TOKEN}'
        },
        params = params
    )
    if res.ok:
        json = res.json()
        row_count = json['count']
        results = json['results']
        if len(results) < row_count:
            print(f'Warning: response got paginated ({len(results)} rows retrieved over a total of {row_count}).')
        return row_count, results
    else:
        print(f'Warning: unsuccessul request (code {res.status_code})!')
        return 0, res.json()

## Notebook offers table

In [15]:
row_count, nb_offers_raw = get_baserow_rows(NB_OFFERS_TABLE, BASEROW_ACCESS_TOKEN)
print(f'Notebook offers table has {row_count} rows')
# print first row
nb_offers_raw[0]

Notebook offers table has 30 rows


{'id': 70,
 'order': '0.50000000000000000000',
 'Offer': 'T470S 256GB Good',
 'Active': True,
 'CPU': [{'id': 67,
   'value': 'Intel Core i5-7300U',
   'order': '1.33333333333333325932'}],
 'Condition': {'id': 2750619, 'value': 'Good', 'color': 'darker-blue'},
 'RAM': '16',
 'SSD': '256',
 'Date de création': '2025-02-22',
 'ITJustGood': 'https://www.itjustgood.com/ultrabook-lenovo-thinkpad-t470s-reconditionne-intel-core-i5-7200u-16go-ddr4-256go-ssd-nvme-thunderbolt-3-windows-10-14-full-hd-tactile-0btd8993.html',
 'Price': '190',
 'Screen resolution': {'id': 2750658, 'value': '1920x1080', 'color': 'green'},
 'Thunderbolt': False,
 'Series': [{'id': 69,
   'value': 'ThinkPad T470S',
   'order': '3.50000000000000000000'}],
 'CPU Age': [{'id': 67, 'value': '8.2'}],
 'GB6 Single': [{'id': 67, 'value': '1088'}],
 'GB6 Multi': [{'id': 67, 'value': '2030'}],
 'GB6 Single/€': [{'id': 67, 'value': '5.7'}],
 'GB6 Multi/€': [{'id': 67, 'value': '10.7'}],
 'Designer': [{'id': 69, 'value': 'Lenovo'

In [19]:
def simplify_nb_offers(table_raw):
    """simplifies the raw notebook offers JSON"""
    table_simple = []
    for row in table_raw:
        row_simple = {
            'Offer': row['Offer'],
            'Price': float(row['Price']),
            'Designer': row['Designer'][0]['value'],
            'Series': row['Series'][0]['value'],
            'CPU': row['CPU'][0]['value'],
            'CPU Age': float(row['CPU Age'][0]['value']),
            'RAM': float(row['RAM']),
            'SSD': float(row['SSD']),
            'Screen resolution': row['Screen resolution']['value'],
            'Condition': row['Condition']['value'],
            'Active': row['Active'],
            'GB6 Single': float(row['GB6 Single'][0]['value']),
            'GB6 Multi': float(row['GB6 Multi'][0]['value']),
            'GB6 Single/€': float(row['GB6 Single/€'][0]['value']),
            'GB6 Multi/€': float(row['GB6 Multi/€'][0]['value']),
            'ITJustGood': row['ITJustGood'],
        }
        table_simple.append(row_simple)
    nrow = len(table_simple)
    print(f'{nrow} notebook offers simplified')
    return table_simple

In [20]:
nb_offers = simplify_nb_offers(nb_offers_raw)
offers_df = pd.DataFrame.from_dict(nb_offers)
offers_df

30 notebook offers simplified


,Offer,Price,Designer,Series,CPU,CPU Age,RAM,SSD,Screen resolution,Condition,Active,GB6 Single,GB6 Multi,GB6 Single/€,GB6 Multi/€,ITJustGood
0,T470S 256GB Good,190.0,Lenovo,ThinkPad T470S,Intel Core i5-7300U,8.2,16.0,256.0,1920x1080,Good,True,1088.0,2030.0,5.7,10.7,https://www.itjustgood.com/ultrabook-lenovo-th...
1,Latitude 5480 240 GB Good,210.0,Dell,Latitude 5480,Intel Core i5-6200U,9.7,16.0,240.0,1920x1080,Good,True,916.0,1832.0,4.4,8.7,https://www.itjustgood.com/ordinateur-portable...
2,Dell Latitude 5400,250.0,Dell,Latitude 5400,Intel Core i5-8365U,6.0,16.0,240.0,1920x1080,Good,True,1270.0,3167.0,5.1,12.7,https://www.itjustgood.com/ordinateur-portable...
3,Dell Latitude 5400,270.0,Dell,Latitude 5400,Intel Core i5-8365U,6.0,16.0,240.0,1366x768,Very Good,True,1270.0,3167.0,4.7,11.7,https://www.itjustgood.com/pc-portable-recondi...
4,Dell Latitude 3400,280.0,Dell,Latitude 3400,Intel Core i5-8365U,6.0,16.0,240.0,1920x1080,Very Good,True,1270.0,3167.0,4.5,11.3,https://www.itjustgood.com/ultrabook-reconditi...
5,Latitude 5410 240GB VGood,400.0,Dell,Latitude 5410,Intel Core i5-10310U,5.0,16.0,240.0,1920x1080,Very Good,True,1297.0,3450.0,3.2,8.6,https://www.itjustgood.com/ordinateur-portable...
6,Lenovo ThinkPad T490S,310.0,Lenovo,ThinkPad T490S,Intel Core i5-8365U,6.0,16.0,500.0,1920x1080,Good,True,1270.0,3167.0,4.1,10.2,https://www.itjustgood.com/ultrabook-lenovo-th...
7,T495S 500GB VGood,350.0,Lenovo,ThinkPad T495S,AMD Ryzen 5 PRO 3500U,6.2,16.0,500.0,1920x1080,Very Good,True,949.0,2620.0,2.7,7.5,https://www.itjustgood.com/ultrabook-reconditi...
8,Dell Latitude 5410,330.0,Dell,Latitude 5410,Intel Core i5-10310U,5.0,16.0,240.0,1366x768,Good,True,1297.0,3450.0,3.9,10.5,https://www.itjustgood.com/ordinateur-portable...
9,Latitude 5410 240GB Good,300.0,Dell,Latitude 5410,Intel Core i5-10310U,5.0,16.0,240.0,1920x1080,Good,True,1297.0,3450.0,4.3,11.5,https://www.itjustgood.com/ordinateur-portable...


In [21]:
offers_df.to_csv('Offers.csv', index=False)

## CPUs table

In [5]:
row_count, cpus_raw = get_baserow_rows(CPUS_TABLE, BASEROW_ACCESS_TOKEN)
print(f'CPUs table has {row_count} rows')
# print first row
cpus_raw[0]

CPUs table has 31 rows


{'id': 133,
 'order': '0.33333333333333331483',
 'Name': 'Intel Core i5-5200U',
 'Geekbench 6 Single-Core': '854',
 'Geekbench 6 Multi-Core': '1611',
 'Architecture': [{'id': 100,
   'value': 'Broadwell',
   'order': '0.50000000000000000000'}],
 'Launch date': '2015-01-01',
 'Product URL': 'https://www.intel.com/content/www/us/en/products/sku/85212/intel-core-i55200u-processor-3m-cache-up-to-2-70-ghz/specifications.html',
 'Comment': '',
 'Cores': '2',
 'P Cores': None,
 'Notebook offers': [],
 'NotebookCheck': '',
 'Geekbench 5.5 Single-Core': None,
 'Geekbench 5.5 Multi-Core': None,
 'Age': '10.2',
 'GB5 Source': None,
 'Designer': [{'id': 100, 'value': 'Intel'}],
 'Win11 Compatible': False,
 'Notebook series': []}

In [6]:
def simplify_cpus(table_raw):
    """simplifies the raw notebook offers JSON"""
    table_simple = []
    for row in table_raw:
        row_simple = {
            'Name': row['Name'],
            'Designer': row['Designer'][0]['value'],
            'Launch date': row['Launch date'],
            'Cores': row['Cores'],
            'Launch date': row['Launch date'],
            'Age': row['Age'],
            'Architecture': row['Architecture'][0]['value'],
            'GB6 Single': float(row['Geekbench 6 Single-Core']),
            'GB6 Multi': float(row['Geekbench 6 Multi-Core']),
            'Win11': 'Yes' if row['Win11 Compatible'] else 'No',
            'Product URL': row['Product URL']
            
        }
        table_simple.append(row_simple)
    nrow = len(table_simple)
    print(f'{nrow} cpus simplified')
    return table_simple

In [7]:
cpus = simplify_cpus(cpus_raw)
cpus_df = pd.DataFrame.from_dict(cpus)
#cpus_df['Launch date'] = pd.to_datetime(cpus_df['Launch date']) # date string to datetime. useless to make a CSV
cpus_df

31 cpus simplified


,Name,Designer,Launch date,Cores,Age,Architecture,GB6 Single,GB6 Multi,Win11,Product URL
0,Intel Core i5-5200U,Intel,2015-01-01,2,10.2,Broadwell,854.0,1611.0,No,https://www.intel.com/content/www/us/en/produc...
1,Intel Core i5-5300U,Intel,2015-01-01,2,10.2,Broadwell,908.0,1692.0,No,https://www.intel.com/content/www/us/en/produc...
2,Intel Core i5-6200U,Intel,2015-07-01,2,9.7,Skylake,916.0,1832.0,No,https://www.intel.com/content/www/us/en/produc...
3,Intel Core i5-6300U,Intel,2015-07-01,2,9.7,Skylake,965.0,1910.0,No,https://www.intel.com/content/www/us/en/produc...
4,Intel Core i5-7200U,Intel,2016-10-01,2,8.5,Kaby Lake,1009.0,1987.0,No,https://www.intel.com/content/www/us/en/produc...
5,Intel Core i5-7300U,Intel,2017-01-01,2,8.2,Kaby Lake,1088.0,2030.0,No,https://www.intel.com/content/www/us/en/produc...
6,Intel Core i5-8250U,Intel,2017-07-01,4,7.7,Kaby Lake R,1145.0,3126.0,Yes,https://www.intel.com/content/www/us/en/produc...
7,Intel Core i5-8350U,Intel,2017-07-01,4,7.7,Kaby Lake R,1191.0,3237.0,Yes,https://www.intel.com/content/www/us/en/produc...
8,Intel Core i5-8365U,Intel,2019-04-01,4,6.0,Whiskey Lake,1270.0,3167.0,Yes,https://www.intel.com/content/www/us/en/produc...
9,Intel Core i5-9400H,Intel,2019-04-01,4,6.0,Coffee Lake,1429.0,4347.0,Yes,https://www.intel.com/content/www/us/en/produc...


In [8]:
cpus_df.to_csv('CPUs.csv', index=False)